<h1><center>

![](https://i.imgur.com/jrRGMfx.png)

AI PRACTITIONER COURSE 

ASSIGNMENT - SESSION 6 - Optimization

**&copy; 2020 VTCA-COTAI. Internal Use Only.**

</center></h1>

# 6.1 Theory Review



## 6.1.1 [1 Point] Search in ML

- What is the search issue in ML?
- Vấn đề tìm kiếm trong ML là gì?

**Answer**: 
- Search issue in ML is essentially optimizing ML model, i.e., finding the best model (or function) in the chosen model architecture (or function space) so that performance measure of the selected model on the training dataset (as well as test set) is highest.
- Vấn đề tìm kiếm trong ML là tối ưu mô hình học máy: tìm mô hình (tức là bộ tham số của hàm) tốt nhất trong không gian mô hình có kiến trúc đang được xét, để chuẩn đánh giá trên tập huấn luyện (cũng như khi triển khai sau này) là cao nhất.

## 6.1.2  [1 Point] Gradient Vector

- What is the geometric intuition of gradient vector?
- Ý nghĩa hình học của vector gradient là gì mà ta lại gọi là gradient descent/ascent?

**Answer**: 
- Gradient vector is the direction in parameter space with highest changing speed/rate of function output.
- Ý nghĩa hình học của vector gradient: là hướng trong không gian tham số mà hàm số có tốc độ thay đổi (giá trị đầu ra) nhanh nhất.

## 6.1.3  [1 Point]  Gradient Descent variants 

- What are the main points to differentiate batch GD  vs. stochastic GD vs. Adam?
- Các điểm chính để so sánh 3 biến thể của giải thuật GD?

**Answer**: 
- Batch GD: true gradient, computed for whole dataset, hence computationally expensive (bad for big training set) but provides fast convergence. Stochastic GD: a sample approximating the true gradient, computed on a single (random) input, hence computationally very cheap (good for big training set) but noisy update.  Adam: momentum + individual adaptive learning rate, hence usually a better method. 
- Batch GD: tính gradient thực sự trên cả dataset nên tính toán lâu. Stochastic GS chỉ tính gradient trên 1 điểm dữ liệu nên chỉ là xấp xỉ gradient thực nhưng tính rất nhanh. Adam sử dụng cả "quán tính" momentum lẫn tốc độ cập nhật thay đổi liên tục cho từng tham số nên thường tìm kiếm/tối ưu tốt hơn. 

# 6.2 [3 Points] Coding Practice

**DATASET LINK** https://drive.google.com/file/d/1yAHURtl1lIL_7Rf9oKXdtYID2E9l9Z5-/view?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Trong bài tập này, chúng ta sẽ dùng Deep Neural Network (DNN) cho bài toán Regression, cụ thể là dự đoán giá nhà dựa vào các thông tin như (diện tích, số phòng ngủ, ...)

Dataset được sử dụng là **Ames Housing dataset**, bao gầm 79 features và 1 label là giá bán của ngôi nhà. Tuy nhiên trong bài tập này, dataset đã được rút gọn chỉ còn 10 features thôi.

Ý nghĩa từng features xem thêm ở đây  [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)



In [ ]:
import numpy as np
np.random.seed(1612)
from tensorflow.random import set_seed
set_seed(1612)

## 6.2.1 [1 Point] Chuẩn bị dữ liệu

Đọc dữ liệu từ file CSV

In [ ]:
import pandas as pd

DATA_PATH = '/content/drive/My Drive/COTAI_Training/6. MLP_Tensorflow/data/s6_houseprice.csv'
df = pd.read_csv(DATA_PATH)
print('Shape of dataframe:',df.shape)

Shape of dataframe: (1462, 11)


In [ ]:
df.head(10) #xem 10 dòng đầu tiên của dataframe 

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,SalePrice
0,8450.0,7.0,5.0,856.0,2.0,1.0,3.0,8.0,0.0,548.0,208500.0
1,9600.0,6.0,8.0,1262.0,2.0,0.0,3.0,6.0,1.0,460.0,181500.0
2,11250.0,7.0,5.0,920.0,2.0,1.0,3.0,6.0,1.0,608.0,223500.0
3,9550.0,7.0,5.0,756.0,1.0,0.0,3.0,7.0,1.0,642.0,140000.0
4,14260.0,8.0,5.0,1145.0,2.0,1.0,4.0,9.0,1.0,836.0,250000.0
5,14115.0,5.0,5.0,796.0,1.0,1.0,1.0,5.0,0.0,480.0,143000.0
6,10084.0,8.0,5.0,1686.0,2.0,0.0,3.0,7.0,1.0,636.0,307000.0
7,10382.0,7.0,6.0,1107.0,2.0,1.0,3.0,7.0,2.0,484.0,200000.0
8,6120.0,7.0,5.0,952.0,2.0,0.0,2.0,8.0,2.0,468.0,129900.0
9,7420.0,5.0,6.0,991.0,1.0,0.0,2.0,5.0,2.0,205.0,118000.0


### Kiểm tra NULL

Kiểm tra kiểu dữ liệu của từng cột và có giá trị NaN (Null) hay không

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotArea       1460 non-null   float64
 1   OverallQual   1460 non-null   float64
 2   OverallCond   1460 non-null   float64
 3   TotalBsmtSF   1460 non-null   float64
 4   FullBath      1460 non-null   float64
 5   HalfBath      1460 non-null   float64
 6   BedroomAbvGr  1460 non-null   float64
 7   TotRmsAbvGrd  1460 non-null   float64
 8   Fireplaces    1460 non-null   float64
 9   GarageArea    1460 non-null   float64
 10  SalePrice     1460 non-null   float64
dtypes: float64(11)
memory usage: 125.8 KB


In [ ]:
df.isna().sum() # Xem số lượng null (na - Not available)

LotArea         2
OverallQual     2
OverallCond     2
TotalBsmtSF     2
FullBath        2
HalfBath        2
BedroomAbvGr    2
TotRmsAbvGrd    2
Fireplaces      2
GarageArea      2
SalePrice       2
dtype: int64

Chúng ta sẽ xóa toàn bộ dữ liệu bị null trong dataset

In [ ]:
df = df.dropna()

Kiểm tra xem đã hết Null chưa

In [ ]:
df.isna().sum() # Xem số lượng null (na - Not available)

LotArea         0
OverallQual     0
OverallCond     0
TotalBsmtSF     0
FullBath        0
HalfBath        0
BedroomAbvGr    0
TotRmsAbvGrd    0
Fireplaces      0
GarageArea      0
SalePrice       0
dtype: int64

Xem thống kê của dataset (Generate descriptive statistics)

In [ ]:
df.describe()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,6.099315,5.575342,1057.429452,1.565068,0.382877,2.866438,6.517808,0.613014,472.980137,180921.195890
std,9981.264932,1.382997,1.112799,438.705324,0.550916,0.502885,0.815778,1.625393,0.644666,213.804841,79442.502883
min,1300.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,34900.000000
25%,7553.500000,5.000000,5.000000,795.750000,1.000000,0.000000,2.000000,5.000000,0.000000,334.500000,129975.000000
50%,9478.500000,6.000000,5.000000,991.500000,2.000000,0.000000,3.000000,6.000000,1.000000,480.000000,163000.000000
75%,11601.500000,7.000000,6.000000,1298.250000,2.000000,1.000000,3.000000,7.000000,1.000000,576.000000,214000.000000
max,215245.000000,10.000000,9.000000,6110.000000,3.000000,2.000000,8.000000,14.000000,3.000000,1418.000000,755000.000000


### Chia dữ liệu thành Train và Test

Tách dữ liệu thành X, y

Do dữ liệu ở đây **chỉ là các con số**, ta có thể dùng hàm .values để chuyển dữ liệu thành numpy array

In [ ]:
# TODO
# 1. Chia dữ liệu thành X và y từ np_data
# 2. X gồm 10 cột đầu và y là cột cuối
# 3. in ra shape của X, y

np_data = df.values
X = np_data[:,:-1]
y = np_data[:,-1]
print('Shape of X:', X.shape)
print('Shape of y:', y.shape)


Shape of X: (1460, 10)
Shape of y: (1460,)


Chia dữ liệu thành Train, Test

In [ ]:
# TODO:
# 1. Chia dữ liệu thành train set và test set
# 2. Dùng random_state = 1612 và test_size = 0.3

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1612)

### Normalize

Quan sát dataset từ hàm df.describe() bên trên, ta thấy được các đặc trưng (feature) của X thuộc nhiều range khác nhau. Nên ở đây ta áp dụng StandardScaler để scale chúng về khoảng giá trị có mean = 0 và độ lệch chuẩn = 1

Do giá trị của output cũng nằm trong range rất lớn, ta cũng sẽ áp dụng StandardScaler để scale output y

In [ ]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

# TODO
# 1. Khởi tạo biến feature_scale từ StandardScaler, và fit vào X_train
# 2. Transform X_train và X_test
# 3. Khởi tạo biến output_scale từ StandardScaler, và fit vào y_train
# 4. Transform y_train và y_test

from sklearn.preprocessing import StandardScaler

feature_scale = StandardScaler().fit(X_train)
X_train = feature_scale.transform(X_train)
X_test = feature_scale.transform(X_test)



output_scale = StandardScaler().fit(y_train)
y_train = output_scale.transform(y_train)
y_test = output_scale.transform(y_test)

## 6.2.2 Ôn tập Keras

### Chú ý 1

Ở các bài lab trước, ta compile model bằng lệnh



```
model.compile(loss='', optimizer='adam', metrics=''
```

Khi compile bằng cách trên, optimizer sẽ dùng learning rate mặc định của hệ thống. Để truyền vào learning rate theo ý thích ta phải tự khởi tạo optimizer



```
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, ...

my_optimizer = Adam(learning_rate = 0.01)
# hoặc
my_optimizer = SGD(learning_rate = 0.01, momentum = 0.9)
# hoặc
my_optimizer = RMSprop(learning_rate = 0.01, momentum = 0.9)

model.compile(loss='', optimizer=my_optimizer, metrics='')
```

Các learning_rate phổ biến gồm 0.1 hoặc 0.01 hoặc 0.001

Các momentum phổ biến gồm 0.9 hoặc 0.8 




### Chú ý 2

Layer đầu tiên phải được đính kèm tham số ``input_shape`` để báo cho mô hình biết số lượng đặc trưng (feature) của dữ liệu đầu vào

```
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(16, activation='relu', input_shape=X_train.shape[1:]))
...
model.add(Dense(1, activation='sigmoid'))
```

Để lấy được tham số ``input_shape`` 1 cách chính xác nhất ta có thể dùng ``input_shape=X_train.shape[1:]``




### Chú ý 3




*   Do đây là bài toán Regression, output layer sẽ có dạng

```
model = Sequential()
...
model.add(Dense(1, activation='linear'))
```

*   Các metric phổ biến trong bài toán Regression bao gồm MAE, MSE, RMSE. Để kết hợp nhiều metric ta dùng

```
from tensorflow.keras.metrics import RootMeanSquaredError

model.compile(loss='mse', optimizer='', metrics=['mae',RootMeanSquaredError()])
```










Đọc thêm về các loại metrics cho bài toán Regressor [here](https://vtca-cotai-hcm.slack.com/?redir=%2Ffiles%2FU01ETPXTVV0%2FF01EWNG6DGV%2Fpractitioner_support_class_01.pptx)

### Chú ý 4



Cách vẽ chart sau khi train model



```
model.compile(loss='mse', optimizer='', metrics=['mae',RootMeanSquaredError()])

history = model.fit(X_train, y_train, epochs=50)
```
Lúc này object history sẽ lưu lại toàn bộ 3 thông số 'loss',  'mae'   'root_mean_squared_error' sau 50 epochs

Để xem những thông số được lưu ta dùng lệnh

```
print(history.history.keys())
```

Để vẽ chart cho loss sau 50 epochs, ta dùng lệnh

```
plt.plot(history.history['loss']) # loss ở đây là 1 trong các giá trị được trả về khi gọi hàm print() bên trên
```

## 6.2.3 Kết hợp sklearn GridSearchCV với Keras

**Lưu ý**

Phần bên dưới chỉ là code tham khảo, học viên đọc hiểu code chứ đừng chạy.

Hãy modify và kết hợp phần code bên dưới vào bài làm của mình.

Để kết hợp GridSearchCV từ sklearn vào Keras, ta cần import các thư viện sau

In [ ]:
from sklearn.model_selection import GridSearchCV
# đây là bài toán Regression nên ta dùng KerasRegressor
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

Ta cần viết hàm create_model() để tạo Keras model 

In [ ]:
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.random import set_seed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



def create_model(optimizer='adam', learning_rate=0.1, momentum=0.9):
  set_seed(42)
  model = Sequential()
  model.add(Dense(32, activation='relu', input_shape=X_train.shape[1:]))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='linear'))
  my_optim=None
  if optimizer == 'sgd':
    my_optim = SGD(learning_rate=learning_rate, momentum=momentum)
  elif optimizer == 'rmsprop':
    my_optim = RMSprop(learning_rate=learning_rate, momentum=momentum)
  else:
    # ta dùng adam cho các trường hợp còn lại
    # lưu ý trong trường hợp optimizer là adam sẽ không có tham số momentum
    my_optim = Adam(learning_rate=learning_rate)

  model.compile(loss='mse', optimizer=my_optim, metrics=['mae',RootMeanSquaredError()])
  return model

Tiếp theo ta tạo 1 dictionary chứa các thông số cần search

In [ ]:
# define the grid search parameters
optimizer_values = ['adam', 'sgd', 'rmsprop']
lr_values = [0.001, 0.005, 0.01, 0.05, 0.1]
momentum_values = [0.0, 0.6, 0.8, 0.9]

param_grid = {
    'optimizer': optimizer_values,
    'learning_rate': lr_values,
    'momentum': momentum_values
}

Khởi tạo model bằng KerasRegressor

In [ ]:
model = KerasRegressor(build_fn=create_model, epochs=100, verbose=1)

Khởi tạo GridSearchCV và fit model trên X_train, y_train

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best model using: %s" % (grid_result.best_params_))

Epoch 1/80
37/37 [==============================] - 0s 1ms/step - loss: 0.7014 - mae: 0.5381 - root_mean_squared_error: 0.8375
Epoch 2/80
37/37 [==============================] - 0s 990us/step - loss: 0.2790 - mae: 0.3442 - root_mean_squared_error: 0.5282
Epoch 3/80
37/37 [==============================] - 0s 956us/step - loss: 0.3305 - mae: 0.3016 - root_mean_squared_error: 0.5749
Epoch 4/80
37/37 [==============================] - 0s 932us/step - loss: 0.3453 - mae: 0.3317 - root_mean_squared_error: 0.5877
Epoch 5/80
37/37 [==============================] - 0s 968us/step - loss: 0.3454 - mae: 0.3387 - root_mean_squared_error: 0.5877
Epoch 6/80
37/37 [==============================] - 0s 908us/step - loss: 0.3141 - mae: 0.3201 - root_mean_squared_error: 0.5604
Epoch 7/80
37/37 [==============================] - 0s 967us/step - loss: 0.2656 - mae: 0.3071 - root_mean_squared_error: 0.5154
Epoch 8/80
37/37 [==============================] - 0s 968us/step - loss: 0.2515 - mae: 0.3251 - ro

Lấy ra best model

In [ ]:
best_model = grid_result.best_estimator_

Đánh giá performance trên Test Set

In [ ]:
best_model.model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 1ms/step - loss: 0.1216 - mae: 0.2697 - root_mean_squared_error: 0.3488


[0.12163986265659332, 0.2696669399738312, 0.34876906871795654]

## 6.2.4 [2 Point] Your Implement 

Ở phần này, học viên tự implement 1 Deep Neural Network (DNN) để giải quyết bài toán dự đoán giá nhà

**GOAL**

*   Cần phải nắm vững cách dùng Tensorflow 2 cùng Keras
*   Tự implement 1 mạng DNN để giải quyết bài toàn trên, với số hidden layer và số hidden unit tùy chọn
*   **Áp dụng GridSearchCV trên Keras để tìm ra optimizer cùng learning_rate, momentum (nếu có) tốt nhất**
*   **Sau khi train model xong đánh giá performance trên Test Set**

